In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 16, Finished, Available, Finished)

In [2]:
%pip install reverse-geocoder
import reverse_geocoder as rg

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 9, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... - \ done
  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268064 sha256=25d8148e756d34f2ffc090098fbc6d0dd6b053a2673515145f3edbb8979ce31b
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/17/3c/41/2bc89719586c2a5c53e9a527daa76a968a1288315c1ae2d904
Successfully built reverse-geocoder

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [5]:
coordinates=(33.3003333,-116.2538333)
rg.search(coordinates)

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 13, Finished, Available, Finished)

Loading formatted geocoded file...


[{'lat': '33.25587',
  'lon': '-116.37501',
  'name': 'Borrego Springs',
  'admin1': 'California',
  'admin2': 'San Diego County',
  'cc': 'US'}]

In [4]:
df=spark.read.table('EarthquakeLH.earthquake_events_silver')
display(df)

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6e144e0d-8c1f-41b4-83b9-755e15bdfe92)

In [6]:
def get_country_code(lat, lon):
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 14, Finished, Available, Finished)

In [9]:
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 17, Finished, Available, Finished)

In [10]:
df_with_location = df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 18, Finished, Available, Finished)

In [11]:
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 19, Finished, Available, Finished)

In [12]:
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, a9a76ab5-9074-404b-9b8f-f1a2c7a19f48, 20, Finished, Available, Finished)